In [ ]:
import tensorflow as tf

class CppTfTest(tf.Module):

    def __init__(self, name=None):
        super().__init__(name=name)

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None, None], dtype=tf.float32),
        ])
    def call(self, input):

        frames = tf.range(600, dtype=tf.float32)

        bpm = tf.random.uniform(
            tf.TensorShape([600,]),
            minval=0,
            maxval=90,
            dtype=tf.dtypes.float32
            )

        output = tf.concat(([frames], [bpm]), axis=0)

        return output

In [ ]:
cpp_tf_test = CppTfTest()
tf.saved_model.save(
    cpp_tf_test,
    'cpp_tf_test',
    signatures=cpp_tf_test.call.get_concrete_function()
    )

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('cpp_tf_test')
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

tflite_model = converter.convert()

with open('cpp_tf_test.tflite', 'wb') as f:
  f.write(tflite_model)
  f.close()


In [ ]:
interpreter = tf.lite.Interpreter('cpp_tf_test.tflite')
#my_signature = interpreter.get_signature_runner()
interpreter.allocate_tensors()

In [ ]:
interpreter.invoke()